In [2]:
import pandas as pd
import sqlite3
import numpy as np
import random
import datetime
#import matplotlib as plt
from matplotlib import pyplot as plt

In [46]:
db_yahoo = '/home/ian/Data/yahoo.db'
database_yahoo = sqlite3.connect(db_yahoo)
yahoo_string = '''SELECT * FROM divs;'''

df_yahoo = pd.read_sql_query(yahoo_string, database_yahoo)
                       
db_advfn = '/home/ian/Data/advfn.db'
database_advfn = sqlite3.connect(db_advfn)

advfn_string = '''SELECT company_ticker, report_date, `Basic EPS (Continuing)` as eps FROM quarterly_reports;'''

df_advfn = pd.read_sql_query(advfn_string, database_advfn)

db_tmx = '/home/ian/Data/tmx.db'
database_tmx = sqlite3.connect(db_tmx)

tmx_string = '''SELECT * FROM tmx_earnings;'''

df_tmx = pd.read_sql_query(tmx_string, database_tmx)

price_string = '''SELECT symbol, Date, Close FROM tsx_prices'''
df_price = pd.read_sql_query(price_string, database_yahoo)


In [47]:
df_advfn['symbol'] = df_advfn['company_ticker'].apply(lambda x: x.replace('.', '-'))

In [48]:
df_advfn['symbol'].unique()

array(['AAV', 'ABX', 'AC', 'ACB', 'ACO-X', 'AD', 'AEM', 'AGI', 'AIF',
       'ALA', 'AP-UN', 'APH', 'AQN', 'ARE', 'ARX', 'ASR', 'ATA', 'ATD-B',
       'AX-UN', 'BAD', 'BAM-A', 'BB', 'BBD-B', 'BBU-UN', 'BCB', 'BCE',
       'BEI-UN', 'BEP-UN', 'BHC', 'BIP-UN', 'BIR', 'BLX', 'BMO', 'BNS',
       'BPY-UN', 'BTE', 'BTO', 'BYD-UN', 'CAE', 'CAR-UN', 'CAS', 'CCA',
       'CCL-B', 'CCO', 'CEU', 'CFP', 'CG', 'CGX', 'CHE-UN', 'CHP-UN',
       'CHR', 'CIGI', 'CIX', 'CLS', 'CM', 'CMG', 'CNQ', 'CNR', 'CP',
       'CPG', 'CPX', 'CRR-UN', 'CSH-UN', 'CSU', 'CTC-A', 'CU', 'CUF-UN',
       'CVE', 'CWB', 'D-UN', 'DGC', 'DII-B', 'DOL', 'DOO', 'DRG-UN',
       'DSG', 'ECA', 'ECN', 'EDV', 'EFN', 'EFX', 'EIF', 'ELD', 'EMA',
       'EMP-A', 'ENB', 'ENGH', 'ERF', 'ESI', 'EXE', 'FCR', 'FFH', 'FM',
       'FNV', 'FR', 'FRU', 'FSV', 'FTS', 'FTT', 'FVI', 'G', 'GC', 'GEI',
       'GIB-A', 'GIL', 'GOOS', 'GRT-UN', 'GTE', 'GUD', 'GUY', 'GWO', 'H',
       'HBC', 'HBM', 'HCG', 'HR-UN', 'HSE', 'IAG', 'IFC', 'IFP', 'IGM',

In [49]:
df_yahoo['parsed_date'] = df_yahoo['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_yahoo.head(2)

,symbol,Date,Dividends,parsed_date
0,ABX,2018-12-27,0.07,2018-12-27
1,ABX,2018-11-29,0.03,2018-11-29


In [50]:
df_advfn['parsed_date'] = df_advfn['report_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m'))
df_advfn.head(2)

,company_ticker,report_date,eps,symbol,parsed_date
0,AAV,2017/06,0.10,AAV,2017-06-01
1,AAV,2017/09,0.07,AAV,2017-09-01


In [51]:
df_price['parsed_date'] = df_price['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_price.head(5)

,symbol,Date,Close,parsed_date
0,AAV,2009-10-29,7.13,2009-10-29
1,AAV,2009-10-30,6.76,2009-10-30
2,AAV,2009-11-02,6.25,2009-11-02
3,AAV,2009-11-03,6.74,2009-11-03
4,AAV,2009-11-04,6.73,2009-11-04


In [52]:
df_tmx['parsed_date'] = df_tmx['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
df_tmx.head(5)

,symbol,date,eps,consensus_eps,surprise,parsed_date
0,RCI.B,2019-04-18,0.78,0.94,-17.02,2019-04-18
1,RCI.B,2019-01-24,1.13,1.08,4.63,2019-01-24
2,RCI.B,2018-10-19,1.21,1.16,4.31,2018-10-19
3,RCI.B,2018-07-19,1.07,1.02,4.90,2018-07-19
4,RCI.B,2018-04-19,0.90,0.74,21.62,2018-04-19


In [53]:
df_merge = pd.concat([df_tmx[['symbol', 'parsed_date', 'eps']], df_yahoo[['symbol', 'parsed_date', 'Dividends']], df_price[['symbol', 'parsed_date', 'Close']] ])
df_merge.head(5)

/home/ian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Close,Dividends,eps,parsed_date,symbol
0,NaN,NaN,0.78,2019-04-18,RCI.B
1,NaN,NaN,1.13,2019-01-24,RCI.B
2,NaN,NaN,1.21,2018-10-19,RCI.B
3,NaN,NaN,1.07,2018-07-19,RCI.B
4,NaN,NaN,0.90,2018-04-19,RCI.B


In [54]:
df_merge.sort_values(by = ['symbol','parsed_date','Dividends'], ascending = [True, False, False], inplace = True )


In [55]:
df_bns = df_merge[['symbol', 'parsed_date', 'Close', 'eps', 'Dividends']][df_merge['symbol'] == 'BNS'].copy()


In [72]:
df_bns.fillna(method='ffill', inplace = True)
df_bns['p-e'] = 0
df_bns['yld'] = 0

df_bns

,symbol,parsed_date,Close,eps,Dividends,p-e,yld
1708,BNS,2019-05-28,NaN,0.0000,NaN,0,0
2716,BNS,2019-04-01,NaN,0.0000,0.87,0,0
1709,BNS,2019-02-26,NaN,1.7500,0.87,0,0
142707,BNS,2019-02-15,74.77,1.7500,0.87,0,0
142706,BNS,2019-02-14,74.32,1.7500,0.87,0,0
142705,BNS,2019-02-13,74.59,1.7500,0.87,0,0
142704,BNS,2019-02-12,74.62,1.7500,0.87,0,0
142703,BNS,2019-02-11,74.48,1.7500,0.87,0,0
142702,BNS,2019-02-08,74.48,1.7500,0.87,0,0
142701,BNS,2019-02-07,74.83,1.7500,0.87,0,0


In [80]:
def get_pe_yld(row):
    if row.at['eps'] != 0:
        row.at['p-e'] = row.at['Close'] / row.at['eps']
    if (row.at['Close'] != 0.0) & (row.at['Close'] != np.NaN) & (not isnull(row.at['Close'])):
        row.at['yld'] = row.at['Dividends'] / row.at['Close']

In [81]:
df_bns.apply(get_pe_yld, axis = 1)
df_bns

NameError: ("name 'isnull' is not defined", 'occurred at index 1708')

In [75]:
df_bns.loc[141637]['Close']

'null'